In [43]:
import pyltr
import pandas as pd
import numpy as np
import MRR
import csv
import time

In [44]:
training_file = "../data/training_feature_data.csv"

In [60]:
# training_data = pd.read_csv("../data/training_feature_data.csv", header=None)

start_time = time.time()

preallocate = 5000000

training_ngram_features = []
training_ylabels = []
training_qids = []

training_ngram_features=preallocate*[[0,0,0,0,0,0]]
training_ylabels=preallocate*[np.bool(0)]
training_qids=preallocate*[np.int(0)]

i = 0
with open(training_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        if int(row[3]) % 2 == 0:
            continue
        training_ngram_features[i] = eval(row[0]) 
        training_ylabels[i] = np.bool(row[1])
        training_qids[i] = np.int(row[3])
        i = i + 1
        if i == preallocate:
            break
        
            
training_ngram_ints = np.array(training_ngram_features)
training_ylabels_ready = np.array(training_ylabels)
training_qids_ready = np.array(training_qids)

training_ngram_features = []
training_ylabels = []
training_qids = []

print("Dit koste: " + str(time.time() - start_time))

Dit koste: 67.54274129867554


In [ ]:
validation_file = "../data/validation_feature_data.csv"

In [ ]:
# training_data = pd.read_csv("../data/training_feature_data.csv", header=None)

start_time = time.time()

preallocate = 5000000

training_ngram_features = []
training_ylabels = []
training_qids = []

training_ngram_features=preallocate*[[0,0,0,0,0,0]]
training_ylabels=preallocate*[np.bool(0)]
training_qids=preallocate*[np.int(0)]

i = 0
with open(training_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        if int(row[3]) % 2 == 0:
            continue
        training_ngram_features[i] = eval(row[0]) 
        training_ylabels[i] = np.bool(row[1])
        training_qids[i] = np.int(row[3])
        i = i + 1
        if i == preallocate:
            break
        
            
training_ngram_ints = np.array(training_ngram_features)
training_ylabels_ready = np.array(training_ylabels)
training_qids_ready = np.array(training_qids)

training_ngram_features = []
training_ylabels = []
training_qids = []

print("Dit koste: " + str(time.time() - start_time))

In [ ]:
metric = pyltr.metrics.ERR(k=10, highest_score = 1)
metric_MRR = MRR.MRR(k=10, highest_score = 1)

# Only needed if you want to perform validation (early stopping & trimming)
# monitor = pyltr.models.monitors.ValidationMonitor(
#     training_ngram_ints, training_ylabels_ready, training_qids_ready, metric=metric, stop_after=250)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=300,
    verbose=1,
)

model.fit(training_ngram_ints, training_ylabels_ready, training_qids_ready)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.6531      543.41m                                         
    2       0.6531      555.36m                                         
